In [5]:
# Instalar dependências
!pip install tensorflow matplotlib

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
# Carregar dataset de gatos e cachorros
import tensorflow_datasets as tfds
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Pré-processamento
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(1)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(1)
# Carregar MobileNetV2 sem o topo (camada final)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE + (3,),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # Congelar a base

# Adicionar camadas finais
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # 2 classes
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar
history = model.fit(ds_train, validation_data=ds_val, epochs=3)
# Avaliar desempenho
loss, acc = model.evaluate(ds_val)
print(f"Accuracy: {acc*100:.2f}%")


Epoch 1/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 43s 64ms/step - accuracy: 0.9518 - loss: 0.1366 - val_accuracy: 0.9794 - val_loss: 0.0602
Epoch 2/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.9802 - loss: 0.0565 - val_accuracy: 0.9817 - val_loss: 0.0566
Epoch 3/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.9824 - loss: 0.0506 - val_accuracy: 0.9815 - val_loss: 0.0561
146/146 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.9841 - loss: 0.0494
Accuracy: 98.15%
